In this assignment you will work with restricted Boltzmann machines. Always show how you arrived at your answer. Hand in your assignment by adding the solutions to this notebook file.

<H3>Exercise 1 (2 points)</H3>

Consider the free energy for the RBM:
$$
F(\mathbf{v}) = -\mathbf{b}^T\mathbf{v} - \sum_i \log(1 + \exp(c_i + \mathbf{w}_i^T\mathbf{v}))\,.
$$
Derive $\frac{\partial F}{\partial b_j}$, $\frac{\partial F}{\partial c_i}$ and $\frac{\partial F}{\partial w_{ij}}$. Show how you arrive at your answer.

<H3>Exercise 1 solutions</H3>


 $\frac{\partial F}{\partial b_j} = \frac{\partial -\mathbf{b}^T\mathbf{v} - \sum_i \log(1 + \exp(c_i + \mathbf{w}_i^T\mathbf{v}))}{\partial b_j} $ treat the term $ \sum_i \log(1 + \exp(c_i + \mathbf{w}_i^T\mathbf{v}))$ as a constant and ignore it gives $\frac{\partial -\mathbf{b}^T\mathbf{v}}{\partial b_j} = \frac{\partial -(b_1 v_1 + ... + b_j v_j + ... + b_n v_n)}{\partial b_j} = -v_j$
 
$ \frac{\partial F}{\partial c_j} = \frac{\partial -\mathbf{b}^T\mathbf{v} - \sum_i \log(1 + \exp(c_i + \mathbf{w}_i^T\mathbf{v}))}{\partial c_j} = \frac{\partial \log(1 + \exp(c_i + \mathbf{w}_i^T\mathbf{v}))}{\partial c_j} = \frac{\partial \log(x)}{\partial x} \frac{\partial 1 + \exp(c_i + \mathbf{w}_i^T\mathbf{v})}{c_i} = \frac{1}{x} exp(c_i + w_i^T v) = - \frac{exp(c_i + w_i^T v)}{1 + exp(c_i + w^t_i v)}$ which is of the same form as $- sigmoid (c_i + w^t_i v)$ and according to "A Practical Guide to Training Restricted Boltzmann
Machines" page 4, this is the definition of the probability $-p(h_i = 1 | v)$.

$\frac{\partial F}{\partial w_{ij}} = \frac{\partial -\mathbf{b}^T\mathbf{v} - \sum_i \log(1 + \exp(c_i + \mathbf{w}_i^T\mathbf{v}))}{\partial w_{ij}} = \frac{\partial \log(1 + \exp(c_i + \mathbf{w}_i^T\mathbf{v}))}{\partial w_{ij}} = \frac{\partial \log(x)}{\partial x} \frac{\partial 1 + \exp(c_i + \mathbf{w}_i^T\mathbf{v})}{w_{ij}} = \frac{1}{x} exp(c_i + w_i^T v) = - \frac{exp(c_i + w_i^T v)}{1 + exp(c_i + w^t_i v)}$ which is again of the same form as $-sigmoid (c_i + w^t_i v)$ except now the term $w^t_i$ is not treated as a constant so the multiplication with $v$ matters and this is the definition of the probability $-p(h_i = 1 | v)v_j$.

<H3>Exercise 2 (8 points)</H3>

Restricted Boltzmann machines (RBMs) form the basis for deep belief networks. In this practical exercise you will implement the RBM yourself.

We included a dataset digits.mat. This contains a subset of the MNIST digit dataset, discretized to binary values. It contains 10000 digits of size 28 x 28.

In [ ]:
import scipy.io
mat = scipy.io.loadmat('digits.mat')
    
X = mat['digits'].astype('float32')

Implement the learning algorithm for restricted Boltzmann machines. Try to keep the algorithm as simple as possible (i.e., don’t implement all the various tricks mentioned in the paper). Use *RBM_train* as a starting point. Apply the RBM learning algorithm to the digits dataset.

In [ ]:
import numpy as np

def gibbs(x, W, b, ngibbs=20, temp=1.0):

    M = x.size # number of variables
    
    # perform Gibbs sampling
    XM = np.zeros([ngibbs,M])
    XM[0] = x
    for t in range(1,ngibbs):
        for i in range(M):
            pi = sigmoid((np.dot(W[i,:],x) + b[i])/temp)
            x[i] = np.random.rand() < pi
        XM[t,:] = x
    
    return XM

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def RBM_train(X,N=10,T=10):
    # RBM_TRAIN trains a restricted Boltzmann machine on M x nexamples input
    # data X with N hidden units for T time steps.
    #
    # Returns N x M weights W, M x 1 visible unit bias b and N x 1 hidden unit bias c

    M = X.shape[0]
    W = 10**-1*np.random.normal(size=[N,M])
    b = 10**-1*np.random.normal(size=M)
    c = np.zeros(N)
    
    # iterate over examples (iterating over minibatches would be more elegant!)
    for idx in range(T):

        prm = np.random.permutation(X.shape[1])
        for i in prm:
            
            # Implementation of the CD-1 algorithm
            gibbs(X, W, b)
            

    return W,b,c

In [ ]:
W, b, c = RBM_train(X)

Plot the rows of W as receptive fields.

You will now generate fantasies. RBMs are so-called generative models. This means that they can be used to generate visible states. This can be done by running Gibbs sampling for T≫1 steps (instead of just one step in CD-1) from a random initial state of the hidden units. Implement a function *RBM_test(W, b, c, T)* which implements this procedure and visualizes the state of the visible units during Gibbs sampling. Add five different fantasies to your pdf. Reflect on the generated fantasies and how this relates to the learnt parameters.